# Exercício de limpeza de dados em Python

## Regras:

- Aparência: sol, nublado, chuva
- Temperatura: -130 ~ 130 F
- Umidade: 0 ~ 100
- Jogar: sim/não 
- Tratar valores NaN

### Importar bibliotecas

In [2]:
import pandas as pd
import seaborn as srn
import statistics as sts